### Inputs and outputs

In [15]:
#Specify input paths

#WatDiv
# s2rdf_fl = open("/home/amadkour/relatedsys/S2RDF_v1.1-spark2.1/QueryTranslator/WatDivQuerySet/RESULTS-WORKLOAD-test1-5K/resultTimes.txt")
# worq_file_adaptive = open("/home/amadkour/projects/worq/test/output/dbout-1B/results-worq-1B-adaptive.txt")
# results_dir = "results_watdiv"

#LUBM
s2rdf_fl = open("/home/amadkour/relatedsys/S2RDF_v1.1-spark2.1/QueryTranslator/WatDivQuerySet/RESULTS-WORKLOAD-lubm-1K/resultTimes-1-2.txt")
worq_file_adaptive = open("/home/amadkour/projects/worq/test/output/dbout-lubm/results-worq-1K-reduced.txt")
results_dir = "results_lubm-1K"

In [16]:
import collections

####### S2RDF ############
time_s2rdf = list()
results_s2rdf = dict()
all_s2rdf_results = list()

for line in s2rdf_fl:
    line = line.strip()
    parts = line.split('\t')
    id = int(parts[0])
    parts[0] = id
    time = float(parts[1].split(" [SO-OS-SS-VP__")[0])
    parts[1] = time
    time_s2rdf.append(time)
    all_s2rdf_results.append(parts)
    results_s2rdf[id] = time
    
    
######## WORQ-ADAPTIVE #############
time_worq_adaptive = list()
results_worq_adaptive = dict()
all_worq_results_adaptive = list()

for line in worq_file_adaptive:
    line = line.strip()
    parts = line.split('\t')
    id = int(parts[0].split("__VP_SO-OS-SS-VP")[0])
    pattern_id = (((id - 1) / 100) + 1)
    parts[0] = id
    time = float(parts[2])
    parts[2] = time
    all_worq_results_adaptive.append(parts)
    time_worq_adaptive.append(time)
    results_worq_adaptive[id] = time
    
    
results_s2rdf = collections.OrderedDict(sorted(results_s2rdf.items()))
results_worq_adaptive = collections.OrderedDict(sorted(results_worq_adaptive.items()))


#### Workload Query Execution Time

In [17]:
import collections
    
time_s2rdf.sort(reverse=True)
#time_worq.sort(reverse=True)
time_worq_adaptive.sort(reverse=True)

query_execution_out = open("/home/amadkour/projects/worq/"+results_dir+"/queryexecution-sorted.csv","w")

query_execution_out.write("CARD\tS2RDF\n")
for a,b in zip(time_worq_adaptive,time_s2rdf):
    query_execution_out.write(str(a) +"\t"+ str(b)+"\n")
    
query_execution_out.close()

#### Calculate Patterns mean execution time

In [18]:
def meanExecutionTime(entries):    
    id_dict = dict()
    mean_time_dict = dict()
    for key,value in entries.iteritems():
        id = int(key)
        pattern_id = (((id - 1) / 100) + 1)
        time = float(value)
        if pattern_id in id_dict:
            lst = id_dict[pattern_id]
            lst.append(time)
            id_dict[pattern_id] = lst
        else:
            lst = list()
            lst.append(time)
            id_dict[pattern_id] = lst

    for key in id_dict.keys():
        lst = id_dict[key]
        meantime = 0
        size = len(lst)
        for elem in lst:
            meantime += elem
        mean_time_dict[key] = meantime / size

    result = dict()
    for key in mean_time_dict.keys():
        result[key] = mean_time_dict[key]
    
    return collections.OrderedDict(sorted(result.items()))

#### Pattern Mean Execution Time

In [19]:
#mean_worq = meanExecutionTime(results_worq)
mean_worq_adaptive = meanExecutionTime(results_worq_adaptive)
mean_s2rdf = meanExecutionTime(results_s2rdf)

mean_pqe = open("/home/amadkour/projects/worq/"+results_dir+"/mean-patternexecution.csv","w")

mean_pqe.write("CARD\tS2RDF\n")
for a,b in zip(mean_worq_adaptive, mean_s2rdf):
    mean_pqe.write(str(mean_worq_adaptive[a]) +"\t" +str(mean_s2rdf[b])+"\n")

mean_pqe.close()

#### Pattern Mean Query Execution Time (SORTED)

In [20]:
mean_pqe_sorted = open("/home/amadkour/projects/worq/"+results_dir+"/mean-patternexecution-sorted.csv","w")

# #WORQ
# mean_times_worq = list()
# for key,val in mean_worq.iteritems():
#     mean_times_worq.append(float(val))
# mean_times_worq.sort()

#WORQ-ADAPTIVE
mean_times_worq_adaptive = list()
for key,val in mean_worq_adaptive.iteritems():
    mean_times_worq_adaptive.append(float(val))
mean_times_worq_adaptive.sort()

#S2RDF
mean_times_s2rdf = list()
for key,val in mean_s2rdf.iteritems():
    mean_times_s2rdf.append(float(val))
mean_times_s2rdf.sort()

mean_pqe_sorted.write("CARD\tS2RDF\n")
for a,b in zip(mean_times_worq_adaptive,mean_times_s2rdf):
    mean_pqe_sorted.write(str(a)+"\t"+str(b)+"\n")

mean_pqe_sorted.close()

#### Identify number of triples versus mean execution time

In [21]:
###### WORQ-ADAPTIVE #######
numtriples_exectime_adaptive = dict()
for entry in all_worq_results_adaptive:
    numtriples = entry[7]
    if numtriples in numtriples_exectime_adaptive:
        lst = numtriples_exectime_adaptive[numtriples]
        time = float(entry[2])
        lst.append(time)
        numtriples_exectime_adaptive[numtriples] = lst
    else:
        lst = list()
        time = float(entry[2])
        lst.append(time)
        numtriples_exectime_adaptive[numtriples] = lst

# Calculate the average for every pattern
numtriples_worq_results_adaptive = dict()
for key,val in numtriples_exectime_adaptive.iteritems():
    size = len(val)
    total = 0.0
    for v in val:
        total += v
    avg = total/size
    numtriples_worq_results_adaptive[int(key)] = float(avg)
ordered_numtriples_worq_adaptive = collections.OrderedDict(sorted(numtriples_worq_results_adaptive.items()))

#### S2RDF - Get the number of triples for S2RDF from WORQ
id_numtriples_map = dict()
for val in all_worq_results_adaptive:
    id_numtriples_map[val[0]] = val[7]
    
numtriples_id_map_s2rdf = dict()
numtriples_exectime = dict()
for entry in all_s2rdf_results:
    numtriples = id_numtriples_map[entry[0]]
    if numtriples in numtriples_exectime:
        lst = numtriples_exectime[numtriples]
        time = float(entry[1])
        lst.append(time)
        numtriples_exectime[numtriples] = lst
    else:
        lst = list()
        time = float(entry[1])
        lst.append(time)
        numtriples_exectime[numtriples] = lst

#Calculate the average for every pattern
numtriples_results = dict()
for key,val in numtriples_exectime.iteritems():
    size = len(val)
    total = 0.0
    for v in val:
        total += v
    avg = total/size
    numtriples_results[int(key)] = float(avg)
    
ordered_numtriples_s2rdf = collections.OrderedDict(sorted(numtriples_results.items()))

### Save Results

numtriples_out = open("/home/amadkour/projects/worq/"+results_dir+"/mean-numtriples.csv","w")
numtriples_out.write("NumTriples\tCARD\tS2RDF\n")
for a,b in zip(ordered_numtriples_worq_adaptive,ordered_numtriples_s2rdf):
    numtriples_out.write(str(a)+"\t"+str(ordered_numtriples_worq_adaptive[a])+ "\t" + str(ordered_numtriples_s2rdf[b])+"\n")
numtriples_out.close()

#### Identify the queries that ran on cold cache in our system

In [22]:
count = 0
cold_ids = list()
for entry in all_worq_results_adaptive:
    parts = entry
    if parts[6] == "false":
        cold_ids.append(parts[0])
        count += 1

print count

11


#### Get the corresponding queries from S2RDF

In [23]:
s2rdf_cold_matches = list()
for entry in all_s2rdf_results:
    if entry[0] in cold_ids:
        s2rdf_cold_matches.append(entry[1])

print len(s2rdf_cold_matches)

11


#### Save the cold queries to files for both systems

In [24]:
# WORQ-ADAPTIVE
list_worq_cold_ids_adaptive = list()
for entry in all_worq_results_adaptive:
    id = entry[0]
    if id in cold_ids:
        list_worq_cold_ids_adaptive.append(float(entry[2]))
        
list_worq_cold_ids_adaptive.sort(reverse=True)


#S2RDF
list_s2rdf_cold_ids = list()
for entry in all_s2rdf_results:
    id = entry[0]
    if id in cold_ids:
        list_s2rdf_cold_ids.append(float(entry[1]))
        
list_s2rdf_cold_ids.sort(reverse=True)

cold_out = open("/home/amadkour/projects/worq/"+results_dir+"/cold.csv","w")
cold_out.write("CARD\tS2RDF\n")
for a,b in zip(list_worq_cold_ids_adaptive,list_s2rdf_cold_ids):
    cold_out.write(str(a)+"\t"+str(b)+"\n")
cold_out.close()

#### Prepare number of joins versus execution time for both systems

In [25]:
joinstoids = dict()
numjoins = dict()
for parts in all_worq_results_adaptive:
    id  = parts[0]
    if id not in cold_ids:
        if parts[5] in numjoins:
            lst = numjoins[parts[5]]
            lst.append(id)
            numjoins[parts[5]]=lst
        else:
            lst = list()
            lst.append(id)
            numjoins[parts[5]] = lst

In [26]:
# Calculate the average scores per join pattern in WORQ-Adaptive
results_join_worq_adaptive = dict()
for key,val in numjoins.iteritems():
    numelems = float(len(val))
    total = 0
    for v in val:
        total += float(results_worq_adaptive[v])
    avg = total/numelems
    results_join_worq_adaptive[key] = avg

results_join_worq_adaptive_sorted = collections.OrderedDict(sorted(results_join_worq_adaptive.items()))

# Calculate corresponding average for S2RDF
results_join_s2rdf = dict()
for key,val in numjoins.iteritems():
    numelems = float(len(val))
    total = 0
    for v in val:
        total += float(results_s2rdf[v])
    avg = total/numelems
    results_join_s2rdf[key] = avg
    
results_join_s2rdf_sorted = collections.OrderedDict(sorted(results_join_s2rdf.items()))

numjoins_out = open("/home/amadkour/projects/worq/"+results_dir+"/numjoins.csv","w")
numjoins_out.write("NumJoins\tCARD\tS2RDF\n")
for a,b in zip(results_join_worq_adaptive_sorted,results_join_s2rdf_sorted):
    numjoins_out.write(str(a)+"\t"+str(results_join_worq_adaptive_sorted[a])+"\t"+str(results_join_s2rdf_sorted[b])+"\n")
                       
numjoins_out.close()